In [ ]:
%load_ext autoreload
%autoreload 2
import os
import re
import numpy as np
import pandas as pd
import LocationTracking_Functions as lt
import holoviews as hv

In [ ]:
dpath = ""
params = {
    'video_dict': {
        'fps': 30,
        'start': 0,
        'end': None
    },
    'stretch': {
        'height': 1,
        'width': 1
    },
    'tracking_params': {
        'loc_thresh': 99,
        'use_window': True,
        'window_size': 100,
        'window_weight': 0,
        'SIGMA': 1
    }
}

In [ ]:
def track_loc(video_dict, stretch, tracking_params):
    image, crop, video_dict = lt.LoadAndCrop(video_dict, stretch)
    ref = lt.Reference(video_dict, crop, num_frames=100)
    location = lt.TrackLocation(video_dict, tracking_params, ref, crop)
    return location

In [ ]:
loc_ls = []
for cur_fd, dirs, files in os.walk(dpath):
    cur_params = params
    cur_params['video_dict']['dpath'] = cur_fd
    vid_pattern = re.compile(r"^(?P<session>s[0-9]{2}_.*_.*)_r(?P<run>[0-9]) Box (?P<box>[1-4])\.wmv$")
    match_ls = list(filter(None, map(vid_pattern.match, files)))
    for vid_ma in match_ls:
        print("processing {}".format(vid_ma.group()))
        cur_params['video_dict']['file'] = vid_ma.group()
        cur_loc = track_loc(**cur_params)
        sess, run, box = vid_ma.group('session'), vid_ma.group('run'), vid_ma.group('box')
        cur_loc['session'], cur_loc['run'], cur_loc['box'] = sess, run, box
        loc_ls.append(cur_loc)

In [ ]:
loc_master = pd.concat(loc_ls, axis='rows')
loc_master.to_pickle(os.path.join(dpath, "locations.pkl"))